In [ ]:
"""
Importing in requirements for SVGP Pytorch
"""
import tqdm
import math
import torch
import gpytorch

"""
Importing in libraries for SGD-SS-GP
"""
# Requirements for algorithms
import numpy as np
from functools import partial
import time
from IPython.display import clear_output
import inspect
import matplotlib.pyplot as plt

# Make plots inline
%matplotlib inline

"""
Importing algorithm functions
"""
import os
os.chdir('C:/Users/hughw/Documents/MSC project/GP algorithms/Master function files')
from GP_funcs_ZTMFSS import kernel_funcs
from GP_funcs_ZTMFSS import model_funcs
from GP_funcs_ZTMFSS import draw_GP
from GP_funcs_ZTMFSS import fit
from GP_funcs_ZTMFSS import diagnostics
from GP_funcs_ZTMFSS import simulations
from functools import partial
os.chdir('C:/Users/hughw/Documents/MSC project/Simulation results')

"""
Defining procedure to run SVGP
"""

from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy
from torch.utils.data import TensorDataset, DataLoader

def SVGP_train(y, X, lengthscale_init = 1, num_inducing=100, epochs=100, batch_size=100, learn_rate_variational = 0.1, learn_rate_hyper = 0.01, tol = 1e-4,seed=0, min_epochs = 100
              , alpha = 0.05):
    
    # setting dimensions
    ntrain,ntest, p = len(y), len(X), len(X.T)
    
    # loading in data
    train_dataset = TensorDataset(X, y)
    train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
    
    losses = np.zeros(1)

    # Creating model
    class GPModel(ApproximateGP):
        def __init__(self, inducing_points):
            variational_distribution = gpytorch.variational.NaturalVariationalDistribution(inducing_points.size(0))
            variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
            super(GPModel, self).__init__(variational_strategy)
            self.mean_module = gpytorch.means.ConstantMean()
            self.covar_module = gpytorch.kernels.ScaleKernel(base_kernel=gpytorch.kernels.RBFKernel(ard_num_dims = p))

            # Initialize lengthscale
            if np.any(lengthscale_init):
                self.covar_module.base_kernel.lengthscale *= lengthscale_init

        def forward(self, x):
            mean_x = self.mean_module(x)
            covar_x = self.covar_module(x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    np.random.seed(seed)
    inducing_points = X[np.random.choice(ntrain,num_inducing,False), :]
    model = GPModel(inducing_points=inducing_points)
    likelihood = gpytorch.likelihoods.GaussianLikelihood()

    if torch.cuda.is_available():
        model = model.cuda()
        likelihood = likelihood.cuda()

        num_epochs = epochs

    # Setting up model training
    t = time.time()
    torch.manual_seed(seed)
    model.train()
    likelihood.train()
    
    variational_ngd_optimizer = gpytorch.optim.NGD(model.variational_parameters(), num_data=train_y.size(0), lr=learn_rate_variational)

    hyperparameter_optimizer = torch.optim.Adam([
        {'params': model.hyperparameters()},
        {'params': likelihood.parameters()},
    ], lr=learn_rate_hyper)

    mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_y.size(0))
    
    # Setting up convergence criteria
    epochs_iter = tqdm.notebook.tqdm(range(epochs), desc="Epoch")
    i = 0
    param_diff=1
    loss_diff=1
    param = 1/model.covar_module.base_kernel.lengthscale
    
    # Training model
    while (i < epochs and loss_diff>0) or i<min_epochs:
        
        # Within each iteration, we will go over each minibatch of data
        minibatch_iter = tqdm.notebook.tqdm(train_loader, desc="Minibatch", leave=False)
        for x_batch, y_batch in minibatch_iter:
            
            ### Perform NGD step to optimize variational parameters
            variational_ngd_optimizer.zero_grad()
            output = model(x_batch)
            loss = -mll(output, y_batch)
            minibatch_iter.set_postfix(loss=loss.item())
            loss.backward()
            variational_ngd_optimizer.step()

            ### Perform Adam step to optimize hyperparameters
            hyperparameter_optimizer.zero_grad()
            output = model(x_batch)
            loss = -mll(output, y_batch)
            loss.backward()
            hyperparameter_optimizer.step()
        
        # Update convergence criteria
        i+=1
        param_old = param
        param = 1/model.covar_module.base_kernel.lengthscale
        param_diff = np.mean(np.abs((param-param_old).detach().numpy()))
        if i==1:
            loss_new = loss.item()
        else:
            loss_old = loss_new
            loss_new = loss.item()*alpha+(1-alpha)*loss_old
            loss_diff = loss_old - loss_new
        
        losses = np.append(losses,loss.item())
        
        numprint = min(10,p)
        print(model.covar_module.base_kernel.lengthscale.detach().numpy()[0][:numprint])
        print(loss_new, param_diff)
    print("Runtime is ", time.time()-t)
    
    return model, loss, likelihood, losses

def SVGP_test(model,likelihood, ytest, Xtest, batch_size=100):

    test_dataset = TensorDataset(Xtest, ytest)
    test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

    # Getting model evaluations
    model.eval()
    likelihood.eval()
    means = torch.tensor([0.])
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            preds = model(x_batch)
            means = torch.cat([means, preds.mean.cpu()])
    means = means[1:]
    
    print('Test MSE: {}'.format(torch.mean(torch.abs(means - ytest.cpu())**2)))
    
    return means

"""
Defining procedure to run SGP
"""

from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, RBFKernel, InducingPointKernel
from gpytorch.distributions import MultivariateNormal

def SGP_train(y, X, lengthscale_init = 1, num_inducing=100, iterations=100, learn_rate = 0.1, tol = 1e-4, seed=0, min_iterations = 100, alpha = 0.05):
    
    # setting dimensions
    ntrain,ntest, p = len(y), len(X), len(X.T)
    
    losses = np.zeros(1)

    # Creating model
    
    class GPRegressionModel(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood, inducing_points):
            super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
            self.mean_module = ConstantMean()
            self.base_covar_module = ScaleKernel(RBFKernel(ard_num_dims = p))
            self.covar_module = InducingPointKernel(self.base_covar_module, inducing_points=inducing_points, likelihood=likelihood)

            # Initialize lengthscale
            if np.any(lengthscale_init):
                self.base_covar_module.base_kernel.lengthscale *= lengthscale_init

        def forward(self, x):
            mean_x = self.mean_module(x)
            covar_x = self.covar_module(x)
            return MultivariateNormal(mean_x, covar_x)

        np.random.seed(seed)
    
    inducing_points = X[np.random.choice(ntrain,num_inducing,False), :]
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = GPRegressionModel(train_x, train_y, likelihood, inducing_points)

    if torch.cuda.is_available():
        model = model.cuda()
        likelihood = likelihood.cuda()

    # Setting up model training
    t = time.time()
    torch.manual_seed(seed) 
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    
    # Setting up convergence criteria
    i = 0
    param_diff=1
    loss_diff=1
    param = 1/model.base_covar_module.base_kernel.lengthscale
    
    # Training model
    while (i < iterations and loss_diff>0) or i<min_iterations:
        
        # Zero backprop gradients
        optimizer.zero_grad()
        # Get output from model
        output = model(train_x)
        # Calc loss and backprop derivatives
        loss = -mll(output, train_y)
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
        
        # Update convergence criteria
        i+=1
        param_old = param
        param = 1/model.base_covar_module.base_kernel.lengthscale
        param_diff = np.mean(np.abs((param-param_old).detach().numpy()))
        if i==1:
            loss_new = loss.item()
        else:
            loss_old = loss_new
            loss_new = loss.item()*alpha+(1-alpha)*loss_old
            loss_diff = loss_old - loss_new
        
        losses = np.append(losses, loss.item())
        
        numprint = min(10,p)
        if not i % 1:
            print(model.base_covar_module.base_kernel.lengthscale.detach().numpy()[0][:numprint])
            print(loss_new, param_diff)
            print('Iter %d - Loss: %.3f' % (i + 1, loss.item()))
    print("Runtime is ", time.time()-t)
    
    return model, loss, likelihood, losses

def SGP_test(model,likelihood, ytest, Xtest):
    
    # Getting model evaluations
    model.eval()
    likelihood.eval()
    with gpytorch.settings.max_preconditioner_size(10), torch.no_grad():
        preds = model(Xtest)
    
    print('Test MSE: {}'.format(torch.mean(torch.abs(preds.mean - ytest.cpu())**2)))
    
    return preds.mean

In [ ]:
"""
Simulation settings
"""
dlist = [10, 100, 1000, 10000] # toggle
post_fit_svgp = False
batch=1024
minibatch=100
inducing=512
NNpred=True
nns=100
epochs = [100,100,300,300]
iters = [100,100,300,300]
ntrial = 1

modelrun = ["SGP", "SVGP"]

"""
Setting up dimensions and objects for drawing data
"""
n = 10000 # toggle
ntrain = n
ntest = 10000
q=2
corrzz=0.5
corrxz=0.5
r2=0.75
lin = False
block_corr = False
"""
Objects to store results
"""
names = ["SGP", "SVGP"]
nmodel = len(names)
Lengthscales = []
Lambdas = []
Predictions = []
MSerrors_Y = []
MSerrors_F = []
Training_times = []
Testing_times = []
Xtestvals = []
Ytestvals = []
Ftestvals = []
Loss = []


"""
Control panel toggle for training/testing etc.
"""
train = True
test = True
test_store = True
plot = True
train_seed = 1

In [ ]:
for i in range(len(dlist)):
    
    p=dlist[i]
    np.random.seed(8750)
    
    L = np.zeros((nmodel,ntrial,p))
    Lambda = np.zeros((nmodel,ntrial,p))
    Preds = np.zeros((nmodel,ntrial,ntest))
    MSE_F = np.zeros((nmodel, ntrial))
    MSE_Y = np.zeros((nmodel, ntrial))
    Train_time = np.zeros((nmodel, ntrial))
    Test_time = np.zeros((nmodel, ntrial))
    TPR = np.zeros((nmodel, ntrial))
    PPV = np.zeros((nmodel, ntrial))
    Losses = np.zeros((nmodel, ntrial,np.max((epochs[i],iters[i]))+1))

    """
    Looping over iterations of data draws
    """
                   
    for j in range(ntrial):
        np.random.seed(j) # Setting seed to draw data
                   
        """
        Drawing data to use for simulation
        """
        Y,F,X,e,sigma,select,ntrain,ntest = draw_GP.draw_parametric_sin_2d_new2(n, ntest, p-2, 0, 1, corrxz=corrxz,corrzz=corrzz, r2=r2, lin=lin, block_corr = block_corr)

        X = (X-X[:ntrain].mean(0))/X[:ntrain].var(0)**0.5
        X = torch.from_numpy(X).float()
        y = torch.from_numpy(Y.reshape(ntrain+ntest,)).float()
        f = torch.from_numpy(F.reshape(ntrain+ntest,)).float()

        train_x = X[:ntrain, :]
        train_y = y[:ntrain]
        test_x = X[ntrain:, :]
        test_y = y[ntrain:]
        test_f = f[ntrain:]

        Xtrain = np.array(train_x)
        ytrain =np.array(train_y).reshape(len(train_y),1)
        Xtest = np.array(test_x)
        ytest = np.array(test_y).reshape(len(test_y),1)
        ftest = np.array(test_f).reshape(len(test_f),1)

        # Storing relevant X's and y,f for test set
        Xtestvals.append(Xtest[:,:q])
        Ytestvals.append(ytest)
        Ftestvals.append(ftest)

        """
        Running SGP
        """
        if "SGP" in modelrun:
            # Training
            if train:
                t=time.time()
                model,loss,likelihood, losses = SGP_train(train_y, train_x, lengthscale_init = 1 +9*(p>100)+20*(p>1000), num_inducing=inducing, iterations=iters[i], learn_rate = 0.1, 
                                                seed = train_seed, min_iterations=iters[i], alpha = 0.01)
                Train_time[0,j] = time.time()-t
                L[0,j] = 1/model.base_covar_module.base_kernel.lengthscale.detach().numpy()

            # Testing
            if test:
                t = time.time()
                preds = SVGP_test(model,likelihood,test_y, test_x, batch_size=100)

                if test_store:
                    Test_time[0,j] = time.time()-t
                    MSE_Y[0,j] = (torch.mean(torch.abs(preds - test_y.cpu())**2)).detach().numpy()
                    MSE_F[0,j] = (torch.mean(torch.abs(preds - test_f.cpu())**2)).detach().numpy()
                    Preds[0,j] = preds
                    Losses[0,j,:iters[i]+1] = losses
                    print("Latent test function MSE : ", MSE_F[0])
                    print("Test time is : ", Test_time[0])

            # Plotting results
            if plot:
                diagnostics.plot_length_scales_pip(np.concatenate((np.repeat(1,q),np.repeat(0,p-q))),L[0,j],L[0,j]>0.1)
                ztrue = np.reshape(ftest,(int(len(ftest)**0.5),int(len(ftest)**0.5)))
                z = np.reshape(preds.detach().numpy(),(int(len(ftest)**0.5),int(len(ftest)**0.5)))

                x1 =  np.reshape(Xtest[:,0],(int(len(ftest)**0.5),int(len(ftest)**0.5)))
                x2 =  np.reshape(Xtest[:,1],(int(len(ftest)**0.5),int(len(ftest)**0.5)))

                fig,axs = plt.subplots(2,figsize = (10,15))
                z_min, z_max = ztrue.min(), ztrue.max()
                c = axs[0].pcolormesh(x1, x2, z, cmap='RdBu', vmin=z_min, vmax=z_max)
                axs[0].set_title('Predicted test surface')
                axs[0].axis([x1.min(), x1.max(), x2.min(), x2.max()])
                fig.colorbar(c, ax=axs[0])

                z_min, z_max = ztrue.min(), ztrue.max()
                c = axs[1].pcolormesh(x1, x2, ztrue, cmap='RdBu', vmin=z_min, vmax=z_max)
                axs[1].set_title('True test surface')
                axs[1].axis([x1.min(), x1.max(), x2.min(), x2.max()])
                fig.colorbar(c, ax=axs[1])
                plt.show()

                print("Latent test function MSE : ",(torch.mean(torch.abs(preds - test_f.cpu())**2)).detach().numpy())
                print("Observed test data  MSE : ",(torch.mean(torch.abs(preds - test_y.cpu())**2)).detach().numpy())
                
        if "SVGP" in modelrun:
            """
            Running SVGP
            """

            # Training
            if train:
                t=time.time()
                model,loss,likelihood,losses = SVGP_train(train_y, train_x, lengthscale_init = 1+9*(p>100)+20*(p>1000), num_inducing=inducing, epochs=epochs[i], batch_size=batch, learn_rate_variational = 0.01, 
                                                   learn_rate_hyper = 0.01, tol = 1e-3, seed = train_seed, min_epochs = epochs[i], alpha = 0.01)
                Train_time[1,j] = time.time()-t
                L[1,j] = 1/model.covar_module.base_kernel.lengthscale.detach().numpy()

            # Testing
            if test:
                t = time.time()
                preds = SVGP_test(model,likelihood,test_y, test_x, batch_size=100)

                if test_store:
                    Test_time[1,j] = time.time()-t
                    MSE_Y[1,j] = (torch.mean(torch.abs(preds - test_y.cpu())**2)).detach().numpy()
                    MSE_F[1,j] = (torch.mean(torch.abs(preds - test_f.cpu())**2)).detach().numpy()
                    Preds[1,j] = preds
                    Losses[1,j,:epochs[i]+1] = losses
                    print("Latent test function MSE : ", MSE_F[1])
                    print("Test time is : ", Test_time[1])

            # Plotting results
            if plot:
                diagnostics.plot_length_scales_pip(np.concatenate((np.repeat(1,q),np.repeat(0,p-q))),L[1,j],L[1,j]>0.1)
                ztrue = np.reshape(ftest,(int(len(ftest)**0.5),int(len(ftest)**0.5)))
                z = np.reshape(preds.detach().numpy(),(int(len(ftest)**0.5),int(len(ftest)**0.5)))

                x1 =  np.reshape(Xtest[:,0],(int(len(ftest)**0.5),int(len(ftest)**0.5)))
                x2 =  np.reshape(Xtest[:,1],(int(len(ftest)**0.5),int(len(ftest)**0.5)))

                fig,axs = plt.subplots(2,figsize = (10,15))
                z_min, z_max = ztrue.min(), ztrue.max()
                c = axs[0].pcolormesh(x1, x2, z, cmap='RdBu', vmin=z_min, vmax=z_max)
                axs[0].set_title('Predicted test surface')
                axs[0].axis([x1.min(), x1.max(), x2.min(), x2.max()])
                fig.colorbar(c, ax=axs[0])

                z_min, z_max = ztrue.min(), ztrue.max()
                c = axs[1].pcolormesh(x1, x2, ztrue, cmap='RdBu', vmin=z_min, vmax=z_max)
                axs[1].set_title('True test surface')
                axs[1].axis([x1.min(), x1.max(), x2.min(), x2.max()])
                fig.colorbar(c, ax=axs[1])
                plt.show()

                print("Latent test function MSE : ",(torch.mean(torch.abs(preds - test_f.cpu())**2)).detach().numpy())
                print("Observed test data  MSE : ",(torch.mean(torch.abs(preds - test_y.cpu())**2)).detach().numpy())



    """
    Storing results in master lists
    """

    Lengthscales.append(L)
    Lambdas.append(Lambda)
    Predictions.append(Preds)
    MSerrors_Y.append(MSE_Y)
    MSerrors_F.append(MSE_F)
    Training_times.append(Train_time)
    Testing_times.append(Test_time)
    Loss.append(Losses)

In [ ]:
from datetime import date
Output = {"Names" : names, "L" : Lengthscales, "Lambda" : Lambdas,"Preds" : Predictions, "MSE_Y" : MSerrors_Y, "MSE_F" : MSerrors_F, "Train_time" : Training_times, "Test_time" : Testing_times}
String = "EXPERIMENT_{0}_2dsimupto100_{12}_n={1}_ntest={2}_pvals={3}_r2={11}_corr={4}_sigma2={5}_batch={6}_inducing={7}_sgdbatch={8}_nnpred={9}_nnpredsize={10}".format(
    date.today(),ntrain,ntest,dlist, corrxz, np.round(1-r2,1), batch, inducing,minibatch,NNpred,nns, r2, modelrun)
np.save(String, Output) # saving